In [12]:
api_key = 'RGAPI-250186f2-2593-4eb8-aeca-5f952bf9391f'

In [3]:
import requests
import pandas as pd
import numpy as np

api_url = 'https://na1.api.riotgames.com/tft/league/v1/entries/DIAMOND/IV?queue=RANKED_TFT&page=100'

api_url = api_url + '&api_key=' + api_key


In [31]:
d4_response = requests.get(api_url)

In [116]:
# loop through every user in diamond 4 and add their names into a dataframe
page_num = 1
diamond_puuid = pd.DataFrame([])
while True:
    api_url = 'https://na1.api.riotgames.com/tft/league/v1/entries/DIAMOND/IV?queue=RANKED_TFT&page=' + str(page_num)
    api_url = api_url + '&api_key=' + api_key
    response = requests.get(api_url).json()
    if(response != []): 
        df = pd.DataFrame(response)
        df1 = df.loc[:, df.columns.intersection(['puuid'])]
        diamond_puuid = pd.concat([diamond_puuid, df1], axis=0)
    else:
        break

    page_num += 1


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


In [117]:
print(diamond_puuid.head(5))
# print(diamond_summoner_names.tail(5))
print(diamond_puuid.size)

                                               puuid
0  xAHUx3ABXJAB6CJvwk3I4f_fs59yufLEKIZ99o4N5dI0lo...
1  5CpXXSb9wmxCmvl-nJoc97HCqkegjSBtJmHUsJahpv4BXm...
2  OwBCsQUKzIhw11OcYuph1hmltOZAroDVlqFcmnFJwgzRNV...
3  c1Khe6uMdUMGs1JLRlCoMucPiMmlc9vdyAJq7xGvZQQydI...
4  XQnpgeSHnXvLlqDYClsGRdVrP7TQdj9Ir_aTPwi85hylLt...
9496


In [14]:
## get the puuids of the top 250 players in north america

import time

api_url = 'https://na1.api.riotgames.com/tft/league/v1/challenger?queue=RANKED_TFT'
api_url = api_url + '&api_key=' + api_key
challenger_players_response = requests.get(api_url).json()
challenger_summoner_ids = [challenger_players_response['entries'][i]['summonerId'] for i in range(0, len(challenger_players_response['entries']))]
challenger_puuids = []

count = 1
for id in challenger_summoner_ids:
    summoner_url = 'https://na1.api.riotgames.com/tft/summoner/v1/summoners/'+ id + '?api_key=' + api_key
    ## delete when have production key
    if count % 100 == 0:
        time.sleep(120)
        print("sleeping 120s")
    r = requests.get(summoner_url).json()
    challenger_puuids.append(r['puuid'])
    count += 1

sleeping 120s
sleeping 120s


In [209]:
print(len(challenger_puuids))

250


In [113]:
## get a list of all match ids within the last hour
import time
recent_match_ids = []
start_time = int(time.time()) - 36000

##
count = 1
for puuid in challenger_puuids:
    api_url_matches = 'https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/' + puuid + '/ids?start=0&startTime=' + str(start_time) + '&count=20&api_key=' + api_key
    r = requests.get(api_url_matches).json()
    recent_match_ids = list(set(recent_match_ids + r))

    ## delete when have production key
    if count % 100 == 0:
        time.sleep(120)
        print("sleeping 120s")
    count += 1
    

sleeping 120s
sleeping 120s


In [114]:
print(recent_match_ids)

['NA1_4904778104', 'NA1_4904963651', 'NA1_4904690465', 'NA1_4904782933', 'NA1_4904772908', 'NA1_4904731037', 'NA1_4904994599', 'NA1_4904858862', 'NA1_4904917890', 'NA1_4904825284', 'NA1_4904911718', 'NA1_4904828215', 'NA1_4904891453', 'NA1_4904734384', 'NA1_4904701836', 'NA1_4904861920', 'NA1_4904958578', 'NA1_4904800227', 'NA1_4904767540', 'NA1_4904865312', 'NA1_4904715697', 'NA1_4904697911', 'NA1_4904836256', 'NA1_4904680264', 'NA1_4904805704', 'NA1_4904750674', 'NA1_4904878882', 'NA1_4904987134', 'NA1_4904953699', 'NA1_4904953585', 'NA1_4904719680', 'NA1_4904777439', 'NA1_4904713996', 'NA1_4904883892', 'NA1_4904834237', 'NA1_4904945898', 'NA1_4904748758', 'NA1_4905000382', 'NA1_4904838711', 'NA1_4904905695', 'NA1_4904804163', 'NA1_4904757658', 'NA1_4904866451', 'NA1_4904679838', 'NA1_4904882972', 'NA1_4904886684', 'NA1_4904719422', 'NA1_4905010337', 'NA1_4904927945', 'NA1_4904846632', 'NA1_4904910633', 'NA1_4904776457', 'NA1_4904687352', 'NA1_4904960467', 'NA1_4904789352', 'NA1_4904

In [146]:
import numpy as np
## add data to database
augments = pd.DataFrame(columns=['2-1 Games', '2-1 Avg Placement', '3-2 Games', '3-2 Avg Placement', '4-2 Games', '4-2 Avg Placement', 'Total Games', 'Total Avg Placement'])
augments = augments.astype({
    '2-1 Games':int,
    '2-1 Avg Placement':float,
    '3-2 Games':int,
    '3-2 Avg Placement':float,
    '4-2 Games':int,
    '4-2 Avg Placement':float,
    'Total Games':int,
    'Total Avg Placement':float,
    })

data_dragon = requests.get('https://ddragon.leagueoflegends.com/cdn/13.24.1/data/en_US/tft-augments.json').json()
aug_ids = list(data_dragon['data'].keys())
aug_names = [data_dragon['data'][i]['name'] for i in list(data_dragon['data'].keys())]

augments.insert(0, "Augment ID", aug_ids)
augments.insert(0, "Augment Name", aug_names)

augments['2-1 Games'] = 0
augments['3-2 Games'] = 0
augments['4-2 Games'] = 0
augments['Total Games'] = 0


count = 1
for match_id in recent_match_ids:

    if count % 100 == 0:
        time.sleep(120)
        print("sleeping 120s")

    api_url = 'https://americas.api.riotgames.com/tft/match/v1/matches/'+ match_id + '?api_key=' + api_key
    r = requests.get(api_url).json()
    for i in range(0, 8):
        # add to database
        ags = ['', '', '']
        augs = r['info']['participants'][i]['augments']
        for i in range(0, len(augs)):
            ags[i] = augs[i]
        placement = r['info']['participants'][i]['placement']


        if ags[0] != '':
            avg_place_21 = augments.loc[augments['Augment ID'] == ags[0], '2-1 Avg Placement'].values[0]
            games_21 = augments.loc[augments['Augment ID'] == ags[0], '2-1 Games'].values[0]
            avg_total_place_21 = augments.loc[augments['Augment ID'] == ags[0], 'Total Avg Placement'].values[0]
            games_total_21 = augments.loc[augments['Augment ID'] == ags[0], 'Total Games'].values[0]


            if pd.isna(avg_place_21):
                augments.loc[augments['Augment ID'] == ags[0], '2-1 Avg Placement'] = float(placement)
            else:
                augments.loc[augments['Augment ID'] == ags[0], '2-1 Avg Placement'] = ((avg_place_21 * games_21) + placement) / (games_21 + 1)

            if pd.isna(avg_total_place_21):
                augments.loc[augments['Augment ID'] == ags[0], 'Total Avg Placement'] = float(placement)
            else:
                augments.loc[augments['Augment ID'] == ags[0], 'Total Avg Placement'] = ((avg_total_place_21 * games_total_21) + placement) / (games_total_21 + 1)

            augments.loc[augments['Augment ID'] == ags[0], '2-1 Games'] += 1
            augments.loc[augments['Augment ID'] == ags[0], 'Total Games'] += 1

        if ags[1] != '':
            avg_place_32 = augments.loc[augments['Augment ID'] == ags[1], '3-2 Avg Placement'].values[0]
            games_32 = augments.loc[augments['Augment ID'] == ags[1], '3-2 Games'].values[0]
            avg_total_place_32 = augments.loc[augments['Augment ID'] == ags[1], 'Total Avg Placement'].values[0]
            games_total_32 = augments.loc[augments['Augment ID'] == ags[1], 'Total Games'].values[0]

            if pd.isna(avg_place_32):
                augments.loc[augments['Augment ID'] == ags[1], '3-2 Avg Placement'] = float(placement)
            else:
                augments.loc[augments['Augment ID'] == ags[1], '3-2 Avg Placement'] = ((avg_place_32 * games_32) + placement) / (games_32 + 1)
            if pd.isna(avg_total_place_32):
                augments.loc[augments['Augment ID'] == ags[1], 'Total Avg Placement'] = float(placement)
            else:
                augments.loc[augments['Augment ID'] == ags[1], 'Total Avg Placement'] = ((avg_total_place_32 * games_total_32) + placement) / (games_total_32 + 1)

            augments.loc[augments['Augment ID'] == ags[1], '3-2 Games'] += 1
            augments.loc[augments['Augment ID'] == ags[1], 'Total Games'] += 1



        if ags[2] != '':
            avg_place_42 = augments.loc[augments['Augment ID'] == ags[2], '4-2 Avg Placement'].values[0]
            games_42 = augments.loc[augments['Augment ID'] == ags[2], '4-2 Games'].values[0]
            avg_total_place_42 = augments.loc[augments['Augment ID'] == ags[2], 'Total Avg Placement'].values[0]
            games_total_42 = augments.loc[augments['Augment ID'] == ags[2], 'Total Games'].values[0]

            if pd.isna(avg_place_42):
                augments.loc[augments['Augment ID'] == ags[2], '4-2 Avg Placement'] = float(placement)
            else:
                augments.loc[augments['Augment ID'] == ags[2], '4-2 Avg Placement'] = ((avg_place_42 * games_42) + placement) / (games_42 + 1)
            if pd.isna(avg_total_place_42):
                augments.loc[augments['Augment ID'] == ags[2], 'Total Avg Placement'] = float(placement)
            else:
                augments.loc[augments['Augment ID'] == ags[2], 'Total Avg Placement'] = ((avg_total_place_42 * games_total_42) + placement) / (games_total_42 + 1)

            augments.loc[augments['Augment ID'] == ags[2], '4-2 Games'] += 1
            augments.loc[augments['Augment ID'] == ags[2], 'Total Games'] += 1

    count += 1

KeyError: 'info'

In [134]:
augments.to_excel('augments.xlsx')
print(augments.head(5))

print(pd.isna(augments.loc[augments['Augment ID'] == 'TFT6_Augment_TradeSector', '3-2 Avg Placement']))

        Augment Name                   Augment ID  2-1 Games  \
0        Salvage Bin      TFT6_Augment_SalvageBin          0   
1  March of Progress   TFT6_Augment_SlowAndSteady          0   
2     Portable Forge   TFT6_Augment_PortableForge          0   
3     Binary Airdrop   TFT6_Augment_BinaryAirdrop          0   
4     Recombobulator  TFT6_Augment_Recombobulator          0   

   2-1 Avg Placement  3-2 Games  3-2 Avg Placement  4-2 Games  \
0                NaN          0                NaN          0   
1                NaN          0                NaN          0   
2                NaN          0                NaN          0   
3                NaN          0                NaN          0   
4                NaN          0                NaN          0   

   4-2 Avg Placement  Total Games  Total Avg Placement  
0                NaN            0                  NaN  
1                NaN            0                  NaN  
2                NaN            0                  Na

In [70]:
augments = pd.DataFrame(columns=['2-1 Games', '2-1 Avg Placement', '3-2 Games', '3-2 Avg Placement', '4-2 Games', '4-2 Avg Placement', 'Total Games', 'Total Avg Placement'])
data_dragon = requests.get('https://ddragon.leagueoflegends.com/cdn/13.24.1/data/en_US/tft-augments.json').json()
aug_ids = list(data_dragon['data'].keys())
aug_names = [data_dragon['data'][i]['name'] for i in list(data_dragon['data'].keys())]

augments.insert(0, "Augment ID", aug_ids)
augments.insert(0, "Augment Name", aug_names)

augments['2-1 Games'] = 0
augments['3-2 Games'] = 0
augments['4-2 Games'] = 0
augments['Total Games'] = 0

In [72]:
api_url = 'https://americas.api.riotgames.com/tft/match/v1/matches/NA1_4902286636?api_key=RGAPI-250186f2-2593-4eb8-aeca-5f952bf9391f'
r = requests.get(api_url).json()
print(r['info']['participants'][0]['augments'])
print(r['info']['participants'][0]['placement'])

ags = r['info']['participants'][0]['augments']
placement = r['info']['participants'][0]['placement']

avg_place_21 = augments.loc[augments['Augment ID'] == ags[0], '2-1 Avg Placement']
games_21 = augments.loc[augments['Augment ID'] == ags[0], '2-1 Games']
avg_place_32 = augments.loc[augments['Augment ID'] == ags[1], '3-2 Avg Placement']
games_32 = augments.loc[augments['Augment ID'] == ags[1], '3-2 Games']
avg_place_42 = augments.loc[augments['Augment ID'] == ags[2], '4-2 Avg Placement']
games_42 = augments.loc[augments['Augment ID'] == ags[2], '4-2 Games']

avg_total_place_21 = augments.loc[augments['Augment ID'] == ags[0], 'Total Avg Placement']
games_total_21 = augments.loc[augments['Augment ID'] == ags[0], 'Total Games']
avg_total_place_32 = augments.loc[augments['Augment ID'] == ags[1], 'Total Avg Placement']
games_total_32 = augments.loc[augments['Augment ID'] == ags[1], 'Total Games']
avg_total_place_42 = augments.loc[augments['Augment ID'] == ags[2], 'Total Avg Placement']
games_total_42 = augments.loc[augments['Augment ID'] == ags[2], 'Total Games']


if avg_place_21.empty:
    augments.loc[augments['Augment ID'] == ags[0], '2-1 Avg Placement'] = placement
else:
    augments.loc[augments['Augment ID'] == ags[0], '2-1 Avg Placement'] = ((avg_place_21 * games_21) + placement) / (games_21 + 1)

if avg_total_place_21.empty:
    augments.loc[augments['Augment ID'] == ags[0], 'Total Avg Placement'] = placement
else:
    augments.loc[augments['Augment ID'] == ags[0], 'Total Avg Placement'] = ((avg_total_place_21 * games_total_21) + placement) / (games_total_21 + 1)

augments.loc[augments['Augment ID'] == ags[0], '2-1 Games'] += 1
augments.loc[augments['Augment ID'] == ags[0], 'Total Games'] += 1

if avg_place_32.empty:
    augments.loc[augments['Augment ID'] == ags[1], '3-2 Avg Placement'] = placement
else:
    augments.loc[augments['Augment ID'] == ags[1], '3-2 Avg Placement'] = ((avg_place_32 * games_32) + placement) / (games_32 + 1)
if avg_total_place_32.empty:
    augments.loc[augments['Augment ID'] == ags[1], 'Total Avg Placement'] = placement
else:
    augments.loc[augments['Augment ID'] == ags[1], 'Total Avg Placement'] = ((avg_total_place_32 * games_total_32) + placement) / (games_total_32 + 1)

augments.loc[augments['Augment ID'] == ags[1], '3-2 Games'] += 1
augments.loc[augments['Augment ID'] == ags[1], 'Total Games'] += 1

if avg_place_42.empty:
    augments.loc[augments['Augment ID'] == ags[2], '4-2 Avg Placement'] = placement
else:
    augments.loc[augments['Augment ID'] == ags[2], '4-2 Avg Placement'] = ((avg_place_42 * games_42) + placement) / (games_42 + 1)
if avg_total_place_42.empty:
    augments.loc[augments['Augment ID'] == ags[2], 'Total Avg Placement'] = placement
else:
    augments.loc[augments['Augment ID'] == ags[2], 'Total Avg Placement'] = ((avg_total_place_42 * games_total_42) + placement) / (games_total_42 + 1)

augments.loc[augments['Augment ID'] == ags[2], '4-2 Games'] += 1
augments.loc[augments['Augment ID'] == ags[2], 'Total Games'] += 1

['TFT6_Augment_TradeSector', 'TFT9_Augment_ArmyBuilding', 'TFT6_Augment_BinaryAirdrop']
6
    Augment Name                Augment ID  2-1 Games 2-1 Avg Placement  \
83  Trade Sector  TFT6_Augment_TradeSector          2                 6   

    3-2 Games 3-2 Avg Placement  4-2 Games 4-2 Avg Placement  Total Games  \
83          0               NaN          0               NaN            2   

   Total Avg Placement  
83                   6  
